In [1]:
from utils.setting import get_amplify_token
dotenv_path = "../.env"
amplify_token = get_amplify_token(dotenv_path)

In [5]:
from amplify import BinaryPoly, gen_symbols, sum_poly, BinaryQuadraticModel, Solver, decode_solution
from amplify.constraint import equal_to, less_equal, greater_equal
from amplify.client import FixstarsClient

In [3]:
a = [3, 4, 6, 1, 5]
c = [6, 7, 8, 1, 4]

In [4]:
MAX_ITEM = 5

In [6]:
q = gen_symbols(BinaryPoly, MAX_ITEM)

In [7]:
print(q)

[q_0, q_1, q_2, q_3, q_4]


In [12]:
constraint_f = sum_poly(len(a), lambda i: q[i] * a[i])

In [13]:
constraint = less_equal(constraint_f, 12)

In [14]:
print(constraint)

3.000000 q_0 + 4.000000 q_1 + 6.000000 q_2 + q_3 + 5.000000 q_4 <= 12


In [16]:
cost = - sum([q_j * c[i] for i, q_j in enumerate(q)])

In [17]:
print(cost)

- 6.000000 q_0 - 7.000000 q_1 - 8.000000 q_2 - q_3 - 4.000000 q_4


In [32]:
# constraintは絶対発生させる
# その中でlossを最小化したいので、costに0.01を欠ける。
loss = 0.01 * cost + constraint

In [33]:
print(loss.input_poly)
print(loss.input_constraints)

- 0.060000 q_0 - 0.070000 q_1 - 0.080000 q_2 - 0.010000 q_3 - 0.040000 q_4
[(3.000000 q_0 + 4.000000 q_1 + 6.000000 q_2 + q_3 + 5.000000 q_4 <= 12, 1)]


In [34]:
client = FixstarsClient()
client.token = amplify_token
client.parameters.timeout = 1000
client.parameters.outputs.duplicate = True

solver = Solver(client)

In [35]:
result = solver.solve(loss)
len(result)

1

In [36]:
for r in result:
    print(r.energy)
    print(r.values)
    print(decode_solution(q, r.values))



-0.17
{4: 1, 0: 1, 3: 0, 2: 0, 1: 1}
[1, 1, 0, 0, 1]
